In [1]:
import os
os.chdir(r'9 - Rebuild')
import torch
from miditok import REMI, TokenizerConfig  # here we choose to use REMI
from pathlib import Path
import random
from miditok.utils import split_files_for_training
from miditok.data_augmentation import augment_dataset

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

print(f"Using {device}.")

Using cuda.


In [2]:
if device == "cuda":
    print(f"Device: {torch.cuda.get_device_name()}.")

Device: NVIDIA GeForce RTX 4090.


# Data Loading

From our MIDITok research, we know that we we want to 

- Train a BPE tokenizer on the entire dataset.
- Save it / Load it (BPE is deterministic so data doesn't need to be decoded with a the same tokenizer it was encoded with, providing whatever is used was trained on the same data with the same config. Unigram is *not* deterministic however so would require the same exact tokenizer for encode / decode).
- Shuffle file names.
- Split into test / train / validation sets, so songs aren't biased to a set.
- Split the files into chunks for each set.
- Optionally augment the dataset with pitch / velocity / duration shifted versions
- Shuffle the chunks when loading, so that parts of a single song aren't biased to a batch.
- Load the chunks with `max_seq_len` equal to that used when splitting files, to minimise padding / truncated data.
- Split the chunks into context-length sequences and feed them through contiguously.
- Manually reset memories between chunks rather than auto-reset on BOS / EOS tokens.

In [3]:
midi_path = Path(f'../data/midi')
dataset_name = 'vg_large'
midi_dataset_path = Path(f'{midi_path}/{dataset_name}')
relative_midi_file_paths = list(midi_dataset_path.glob("**/*.mid"))
midi_file_paths = [path.resolve() for path in relative_midi_file_paths]

tokenizer_save_path = Path(f'../data/vocab/MidiTok/{dataset_name}.json')

if not tokenizer_save_path.exists():
    TOKENIZER_PARAMS = {
        "pitch_range": (21, 109),
        "beat_res": {(0, 4): 8, (4, 12): 4},
        "num_velocities": 32,
        "use_programs": True
        # "use_chords": True,
        # "use_time_signatures": True,
        # "use_tempos": True,
        # "num_tempos": 32,  # number of tempo bins
        # "tempo_range": (40, 250)
    }
    tokenizer_confg = TokenizerConfig(**TOKENIZER_PARAMS)
    tokenizer = REMI(tokenizer_confg)
    tokenizer.train(vocab_size=1000, files_paths=midi_file_paths)
    tokenizer.save(tokenizer_save_path)
else:
    tokenizer = REMI(params=tokenizer_save_path)

/opt/conda/lib/python3.11/site-packages/miditok/tokenizations/remi.py:77: UserWarning: Attribute controls are not compatible with 'config.one_token_stream_for_programs' and multi-vocabulary tokenizers. Disabling them from the config.
  super().__init__(tokenizer_config, params)


In [4]:
random.seed(42)
random.shuffle(midi_file_paths)
len(midi_file_paths)

3839

In [5]:
n1 = int(0.8 * len(midi_file_paths))
n2 = int(0.9 * len(midi_file_paths))
train_filepaths = midi_file_paths[:n1]
valid_filepaths = midi_file_paths[n1:n2]
test_filepaths = midi_file_paths[n2:]

print(f'Train file names: {len(train_filepaths)}, Valid file names: {len(valid_filepaths)}, Test file names: {len(test_filepaths)}')

Train file names: 3071, Valid file names: 384, Test file names: 384


In [7]:
from multiprocessing import Pool

chunk_path = f'{midi_path}/{dataset_name}_miditok_split'
test_chunk_path = Path(f'{chunk_path}/test')
train_chunk_path = Path(f'{chunk_path}/train')
valid_chunk_path = Path(f'{chunk_path}/valid')

split_data = [
    (train_filepaths, train_chunk_path),
    (valid_filepaths, valid_chunk_path),
    (test_filepaths, test_chunk_path)
]

def chunk_files(filepaths, tokenizer, chunks_dir, max_seq_len):
    split_files_for_training(
        files_paths=filepaths,
        tokenizer=tokenizer,
        save_dir=chunks_dir,
        max_seq_len=max_seq_len,
        num_overlap_bars=1
    )

# Hash file should prevent re-encoding    
with Pool(processes=3) as pool:
    pool.starmap(chunk_files, [(filepaths, tokenizer, chunks_dir, 1024) for filepaths, chunks_dir in split_data])

/tmp/ipykernel_65585/3317919274.py:15: UserWarning: These files have already been split in the saving directory (../data/midi/vg_large_miditok_split/valid). Skipping file splitting.
  split_files_for_training(
/tmp/ipykernel_65585/3317919274.py:15: UserWarning: These files have already been split in the saving directory (../data/midi/vg_large_miditok_split/test). Skipping file splitting.
  split_files_for_training(
/tmp/ipykernel_65585/3317919274.py:15: UserWarning: These files have already been split in the saving directory (../data/midi/vg_large_miditok_split/train). Skipping file splitting.
  split_files_for_training(
